# Intro

Jon Messier

2/20/2023

---

**Business Problem**

For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.

# Part 1

For Part 1 of the project, you will be creating your project repository, loading the official IMDB data for the requested tables, filtering out unnecessary data, and saving the filtered tables as gzip-compressed csv files (".csv.gz") in your repository.

**Getting Started Tips:**

- Please make sure to read the following lesson ["Getting Started - Project 3"](https://login.codingdojo.com/m/376/12528/88061) for additional tips and directions!
    
  **The Data**

- IMDB Provides Several Files with varied information for Movies, TV Shows, Made for TV Movies, etc.
-   Overview/Data Dictionary: https://www.imdb.com/interfaces/
- Downloads page: https://datasets.imdbws.com/

- From their previous research, they realized they want to focus on the following files:
 -  title.basics.tsv.gz
 - title.ratings.tsv.gz
 - title.akas.tsv.gz


**Specifications**

Your stakeholder only wants you to include information for movies based on the following specifications:

-    Exclude any movie with missing values for genre or runtime
-    Include only full-length movies (titleType = "movie").
-    Include only fictional movies (not from documentary genre)
-   Include only movies that were released 2000 - 2021 (include 2000 and 2021)
-    Include only movies that were released in the United States


**Deliverable**

After filtering out movies that do not meet the stakeholder's specifications:

-    Before saving, run a final .info() for each of the dataframes to show a summary of how many movies remain and the datatypes of each feature
-    Save each file to a compressed csv file "Data/" folder inside your repository.
-    Commit your changes to your repository in GitHub desktop and Publish repository / Push Changes.
-    Submit the link to your repository



## Class/Data imports

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'

## Data Inspection and cleanup

### Aka's
- [x]  Replace "\N" with np.nan
- [x] Keep only US movies.

In [ ]:
df_akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
df_akas.info()
df_akas.head()

In [ ]:
#replace \N with np.nan
df_akas = df_akas.replace({'\\N':np.nan})

In [ ]:
#Keep only US movies
df_akas=df_akas.loc[df_akas['region']=="US"]

### Basics
- [x]   Replace "\N" with np.nan
- [x]   Eliminate movies that are null for runtimeMinutes
- [x] Eliminate movies that are null for genre
- [x] keep only titleType==Movie
- [x] keep startYear 2000-2022
- [x]  Eliminate movies that include "Documentary" in genre (see tip below)
- [x]  Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [ ]:
df_basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
df_basics.info()
df_basics.head()

In [ ]:
#replace \N with np.nan
df_basics = df_basics.replace({'\\N':np.nan})
df_basics.info()

In [ ]:
#drop null runtimes
df_basics = df_basics.dropna(axis=0, subset='runtimeMinutes')
df_basics.info()

In [ ]:
#drop null genre
df_basics = df_basics.dropna(axis=0,subset = 'genres')
df_basics.info()

In [ ]:
#keep titletype=movie
df_basics=df_basics[df_basics['titleType']=='movie']
df_basics.info()

In [ ]:
#drop null startYears
df_basics = df_basics.dropna(axis=0,subset = 'startYear')
df_basics.info()

In [ ]:
#convert year to an int
df_basics['startYear']=df_basics['startYear'].astype('int')

#Keep only the movies between 2000-2022
df_basics=df_basics.loc[(df_basics["startYear"]>= 2000) 
                        & (df_basics["startYear"]<= 2022)]
df_basics['startYear'].describe()

In [ ]:
# Exclude movies that are included in the documentary category.
is_documentary = df_basics['genres'].str.contains('documentary',case=False)
df_basics = df_basics[~is_documentary]
df_basics.head()

In [ ]:
# Filter the basics table down to only include the US by using the filter ...
#Akas dataframe
keepers =df_basics['tconst'].isin(df_akas['titleId'])
df_basics = df_basics[keepers]
df_basics.head()

### Ratings

- [x]   Replace "\N" with np.nan
- [x]   Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [ ]:
df_ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
df_ratings.info()
df_ratings.head()

In [ ]:
#replace \N with np.nan
df_ratings = df_ratings.replace({'\\N':np.nan})

In [ ]:
# Filter the ratings table down to only include the US by using the filter ...
#Akas dataframe
keepers =df_ratings['tconst'].isin(df_akas['titleId'])
df_ratings = df_ratings[keepers]
df_ratings.head()

## Data File storage

In [ ]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

In [ ]:
## Save current dataframe to file.
df_basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
df_ratings.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
df_akas.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [ ]:
# Open saved file and preview again
df_basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
df_basics.head()

In [ ]:
# Open saved file and preview again
df_akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
df_akas.head()

In [ ]:
# Open saved file and preview again
df_ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
df_ratings.head()

## File .info() summary

In [ ]:
df_basics.info()

In [ ]:
df_ratings.info()

In [ ]:
df_akas.info()

# Part 2a.
### Your Stakeholder Wants More Data!

-   After investigating the preview of your data from Part 1, your stakeholder realized that there is no financial information included in the IMDB data (e.g. budget or revenue).
 -    This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before you will be able to determine which movies are successful.

-  Your stakeholder identified The Movie Database (TMDB) as a great source of financial data (https://www.themoviedb.org/). Thankfully, TMDB offers a free API for programmatic access to their data!

-  Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

-   **Note: this process can take a long time and may need to run overnight.**

---

### Specifications - Financial Data

- Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

-  As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

-   Each year should be saved as a separate .csv.gz file

**Hint: Use the two custom functions from the lessons (Intro to TMDB API, and Efficient TMDB API Calls). Be sure to define these functions prior to calling them in your code!**

-  One function will add the certification (MPGG Rating) to movie.info
-  The other function will help you append/extend a JSON file with Python

**Confirm Your API Function works.**

In order to ensure your function for extracting movie data from TMDB is working, test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that your function runs without error and that it returns the correct movie's data for both test ids.

**Hint: Ideally you can organize the code segments from the previous lesson to create an outer and inner loop, but if you get stuck, you can complete 1 year at a time.**

-  Once you have retrieved and saved the final results to 2 separate .csv.gz files, move on to a new Exploratory Data Analysis notebook to explore the following questions.


## Class Import, API connection
Import key clases, connect to TMDB api and local data

In [ ]:
import tmdbsimple as tmdb
import pandas as pd
from tqdm.notebook import tqdm_notebook
import os, time, json

with open('/Users/jonme/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

In [ ]:
tmdb.API_KEY =  login['api-key']

## Custom Functions

### `get_movie_with_rating`

In [ ]:
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    
    info = movie.info()
    
    releases = movie.releases()
    
    for c in releases['countries']:
        if c['iso_3166_1']=='US':
            info['certification']=c['certification']
    return info     

In [ ]:
#test get_movie... function with Avengers "tt0848228"
test = get_movie_with_rating("tt0848228") 
test

### `write_json`

In [ ]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

## Load existing data
Load in the Title Basics data

You need to read in the filtered dataframe you created based on the specification of project 3 Part 1.

You will be filtering out the movies for each year inside the loop, so we will need this loaded and ready to be filtered.


In [ ]:
#Check for DATA folder
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

In [ ]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')

**Create Required Lists for the Loop**

Define a list of the Years to Extract from the API

We have data from 2000 - 2020 available. If we just want results for the first two years, we will create a YEARS_TO_GET list. This will control our outer loop.


In [ ]:
YEARS_TO_GET = list(range(2000,2021))

**Define an errors list**

We will want to be able to save the id's and error messages for any movie that causes an error. To do so, we will want to create an empty errors list before our loops that we can append to later.


In [ ]:
errors = [ ]

### Write JSON data to CSV files

In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    print(f'{JSON_FILE} status {file_exists}')
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
        #Saving new year as the current df
        df = basics.loc[ basics['startYear']==YEAR].copy()
        # saving movie ids to list
        movie_ids = df['tconst'].copy()
    
        # Load existing data from json into a dataframe called "previous_df"
        previous_df = pd.read_json(JSON_FILE)
    
        # filter out any ids that are already in the JSON_FILE
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
        #Get index and movie id from list
        # INNER Loop
        for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
            try:
                # Retrieve then data for the movie id
                temp = get_movie_with_rating(movie_id)  
                # Append/extend results to existing file using a pre-made function
                write_json(temp,JSON_FILE)
                # Short 20 ms sleep to prevent overwhelming server
                time.sleep(0.02)
            
            except Exception as e:
                errors.append([movie_id, e])
    
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                         compression="gzip", index=False)